<a href="https://colab.research.google.com/github/amk735/DS340W-project-flower-image-classification/blob/main/FlowerTransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras import optimizers, callbacks
import tensorflow.keras.backend as K

%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU') 
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
image_width = 224
image_height = 224
num_classes = 5
histories = []

In [6]:
images_dir = "flowers-subset/"
train_data_dir = images_dir + "train/"
val_data_dir = images_dir + "val/"
test_data_dir = images_dir + "test/"

In [7]:
images_dir = "flowers-subset/"
flowers_complete = "flowers-complete/"
train_data_dir = images_dir + "train/"
val_data_dir = flowers_complete
test_data_dir = flowers_complete #+ "test/"

In [8]:
checkpoint_dir = "checkpoints/"
checkpoint_name = checkpoint_dir + "flora-transferLearning-{val_loss:.4f}-{val_accuracy:.4f}.hdf5"
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

In [9]:
def create_callbacks():
    return [
        callbacks.EarlyStopping(monitor="val_accuracy", patience=10, verbose=1),

        callbacks.ModelCheckpoint(checkpoint_name, monitor="val_accuracy", 
                                  verbose=1, save_best_only=True),
    ]

my_callbacks = create_callbacks()

In [10]:
my_callbacks = create_callbacks()


In [11]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

base_model = MobileNetV2(input_shape=(image_height, image_width, 3), 
                       include_top=False, weights="imagenet", 
                       pooling=None)

9406464/9406464 [==============================] - 0s 0us/step


In [12]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

train_datagen = ImageDataGenerator(
                    rotation_range=40,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    shear_range=0.2,
                    zoom_range=0.2,
                    channel_shift_range=0.2,
                    horizontal_flip=True,
                    fill_mode="nearest",
                    preprocessing_function=preprocess_input)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [13]:
batch_size = 64

train_generator = train_datagen.flow_from_directory(
                    train_data_dir,
                    target_size=(image_width, image_height),
                    batch_size=batch_size,
                    class_mode="categorical",
                    shuffle=True)

val_generator = val_datagen.flow_from_directory(
                    val_data_dir,
                    target_size=(image_width, image_height),
                    batch_size=batch_size,
                    class_mode="categorical",
                    shuffle=False)

test_generator = test_datagen.flow_from_directory(
                    test_data_dir,
                    target_size=(image_width, image_height),
                    batch_size=batch_size,
                    class_mode="categorical",
                    shuffle=False)

FileNotFoundError: ignored

In [ ]:
from tensorflow.keras import regularizers

top_model = Sequential()
top_model.add(base_model)
top_model.add(GlobalAveragePooling2D())
top_model.add(Dropout(0.7))
top_model.add(Dense(num_classes, kernel_regularizer=regularizers.l2(0.01)))
top_model.add(Activation("softmax"))

In [ ]:
from tensorflow.keras import regularizers

model = Sequential()
model.add(base_model)
model.add(Conv2D(64, (3, 3), input_shape=(image_height, image_width, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#b
model.add(Conv2D(32, (1, 1)))
model.add(Activation('relu'))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, kernel_regularizer=regularizers.l2(0.01)))
model.add(Activation("softmax"))

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

top_model.compile(loss="categorical_crossentropy",
                  optimizer=optimizers.Adam(lr=1e-4),
                  metrics=["accuracy"])    

In [ ]:
histories.append(top_model.fit(train_generator, 
                              steps_per_epoch=len(train_generator),
                              epochs=60,
                              callbacks=my_callbacks,
                              validation_data=val_generator,
                              validation_steps=len(val_generator),
                              workers=8))

Evaluate on the best model

In [ ]:
from tensorflow.keras.models import load_model
model_saved_name = ""

best_model = load_model(checkpoint_dir + "flora-transferLearning-0.3959-0.8959.hdf5")

In [ ]:
best_model.evaluate_generator(test_generator, steps=len(test_generator))


In [ ]:
test_generator.reset()
probabilities = best_model.predict_generator(test_generator, steps=len(test_generator))
predicted_labels = np.argmax(probabilities, axis=-1)

In [ ]:
target_labels = test_generator.classes


In [ ]:
from sklearn import metrics
conf = metrics.confusion_matrix(target_labels, predicted_labels)

In [ ]:
import seaborn as sns

def plot_confusion_matrix(conf, labels, figsize=(8, 8)):
    fig = plt.figure(figsize=figsize)
    heatmap = sns.heatmap(conf, annot=True, fmt="d")
    heatmap.xaxis.set_ticklabels(labels, rotation=45, ha="right", fontsize=12)
    heatmap.yaxis.set_ticklabels(labels, rotation=0, ha="right", fontsize=12)
    plt.xlabel("Predicted label", fontsize=12)
    plt.ylabel("True label", fontsize=12)
    plt.show()

In [ ]:
labels = [""] * num_classes
for k, v in test_generator.class_indices.items():
    labels[v] = k

In [ ]:
plot_confusion_matrix(conf, labels, figsize=(14, 14))


In [ ]:
print(metrics.classification_report(target_labels, predicted_labels, target_names=labels))


In [ ]:
# Find for which images the predicted class is wrong
wrong_images = np.where(predicted_labels != target_labels)[0]

# For every prediction, find the largest probability value;
# this is the probability of the winning class for this image
probs_max = np.max(probabilities, axis=-1)

# Sort the probabilities from the wrong images from low to high
idx = np.argsort(probs_max[wrong_images])

# Reverse the order (high to low), and keep the 5 highest ones
idx = idx[::-1][:5]

# Get the indices of the images with the worst predictions
worst_predictions = wrong_images[idx]

index2class = {v:k for k,v in test_generator.class_indices.items()}

for i in worst_predictions:
    print("%s was predicted as '%s' %.4f\n" % (
        test_generator.filenames[i],
        index2class[predicted_labels[i]],
        probs_max[i]
    ))

In [ ]:
from keras.preprocessing import image
img = image.load_img(test_data_dir + test_generator.filenames[worst_predictions[0]])
plt.imshow(img)

Export to Core ML

In [ ]:
import coremltools as ct
import os
from tensorflow.keras.models import load_model

In [ ]:
label_dir = "ava/"
labels = []
checkpoint_dir = "checkpoints/"
best_model = tensorflow.keras.load_model(checkpoint_dir + "multisnacks-1.6708-0.4054.hdf5")

In [ ]:
for i in os.listdir(label_dir):
    if i != '.DS_Store' and i != '._.DS_Store':
        labels.append(i)

In [ ]:
coreml_model = ct.convert(
    best_model,
    input_names=["image"],
    image_input_names="image",
    output_names="labelProbability",
    predicted_feature_name="label",
    red_bias=-1,
    green_bias=-1,
    blue_bias=-1,
    image_scale=2/255.0,
    class_labels=labels)

In [ ]:
coreml_model.author = "Joshua Ball"
coreml_model.short_description = "Image classifier"

coreml_model.input_description["image"] = "Input image"
coreml_model.output_description["labelProbability"]= "Prediction probabilities"
coreml_model.output_description["label"]= "Class label of top prediction"

In [ ]:
coreml_model.save("MultiSnacks.mlmodel")


In [ ]:
m = ct.models.MLModel("MultiSnacks.mlmodel")


In [ ]:
import numpy as np
import PIL.Image

In [ ]:
Height = 224  # use the correct input image height
Width = 224  # use the correct input image width

In [ ]:
def load_image(path, resize_to=None):
    # resize_to: (Width, Height)
    img = PIL.Image.open(path)
    if resize_to is not None:
        img = img.resize(resize_to, PIL.Image.ANTIALIAS)
    img_np = np.array(img).astype(np.float32)
    return img_np, img

In [ ]:
_, img = load_image('ava/Ripe/focused_184465770-stock-photo-whole-ripe-avocado.jpg', resize_to=(Width, Height))
out_dict = m.predict({'image': img})

In [ ]:
from skimage import io
import matplotlib.pyplot as plt
image = io.imread('a.jpg')

_ = plt.hist(image.ravel(), bins = 256, color = 'orange', )
_ = plt.hist(image[:, :, 0].ravel(), bins = 256, color = 'red', alpha = 0.5)
_ = plt.hist(image[:, :, 1].ravel(), bins = 256, color = 'Green', alpha = 0.5)
_ = plt.hist(image[:, :, 2].ravel(), bins = 256, color = 'Blue', alpha = 0.5)
_ = plt.xlabel('Intensity Value')
_ = plt.ylabel('Count')
_ = plt.legend(['Total', 'Red_Channel', 'Green_Channel', 'Blue_Channel'])
plt.show()

In [ ]:
from mpl_toolkits.axes_grid1 import ImageGrid
fig = plt.figure(figsize=(14., 14.))
grid = ImageGrid(fig, 111,  # similar to subplot(111)
                 nrows_ncols=(1, 5),  # creates 2x2 grid of axes
                 axes_pad=0.0,  # pad between axes in inch.
                 )

image_data = [io.imread('a.jpg'), io.imread('b.jpg'),io.imread('c.jpg'),io.imread('d.jpg'),io.imread('e.jpg')]
for ax, im in zip(grid, image_data):
    # Iterating over the grid returns the Axes.
    ax.imshow(im,interpolation='nearest') 
    ax.set_axis_off()

    plt.axis('off')
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(2, 2, 1)
imgplot = plt.imshow(io.imread('a.jpg'))
ax.set_title('Rose')
#plt.colorbar(ticks=[0.1, 0.3, 0.5, 0.7],  orientation='horizontal')
imgplot.set_clim(0.0, 0.7)

ax = fig.add_subplot(2, 2, 2)
imgplot = plt.imshow(io.imread('b.jpg'))
imgplot.set_clim(0.0, 0.7)
ax.set_title('Dandelion')
#plt.colorbar(ticks=[0.1, 0.3, 0.5, 0.7], orientation='horizontal')

ax = fig.add_subplot(2, 2, 3)
imgplot = plt.imshow(io.imread('c.jpg'))
imgplot.set_clim(0.0, 0.7)
ax.set_title('Dandelion')
#plt.colorbar(ticks=[0.1, 0.3, 0.5, 0.7], orientation='horizontal')

ax = fig.add_subplot(2, 2, 4)
imgplot = plt.imshow(io.imread('d.jpg'))
imgplot.set_clim(0.0, 0.7)
ax.set_title('Sunflower')
#plt.colorbar(ticks=[0.1, 0.3, 0.5, 0.7], orientation='horizontal')

ax = fig.add_subplot(1, 2, 1)
imgplot = plt.imshow(io.imread('e.jpg'))
imgplot.set_clim(0.0, 0.7)
ax.set_title('Tulip')
#plt.colorbar(ticks=[0.1, 0.3, 0.5, 0.7], orientation='horizontal')